<a href="https://colab.research.google.com/github/DeepeshMadane/NLP_Machine_learning/blob/main/kindle_review_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Importing libraries

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
import nltk
import re
import gensim




1.   Data preprocessing and cleaning

2.   Train test split

3.   BOW,Tf-Idf,Word2vec,Avgword2vec

4.   Train with ml model



In [ ]:
# load the dataset
data = pd.read_csv('all_kindle_review.csv')

In [ ]:
data.head()

,Unnamed: 0.1,Unnamed: 0,asin,helpful,rating,reviewText,reviewTime,reviewerID,reviewerName,summary,unixReviewTime
0,0,11539,B0033UV8HI,"[8, 10]",3,"Jace Rankin may be short, but he's nothing to ...","09 2, 2010",A3HHXRELK8BHQG,Ridley,Entertaining But Average,1283385600
1,1,5957,B002HJV4DE,"[1, 1]",5,Great short read. I didn't want to put it dow...,"10 8, 2013",A2RGNZ0TRF578I,Holly Butler,Terrific menage scenes!,1381190400
2,2,9146,B002ZG96I4,"[0, 0]",3,I'll start by saying this is the first of four...,"04 11, 2014",A3S0H2HV6U1I7F,Merissa,Snapdragon Alley,1397174400
3,3,7038,B002QHWOEU,"[1, 3]",3,Aggie is Angela Lansbury who carries pocketboo...,"07 5, 2014",AC4OQW3GZ919J,Cleargrace,very light murder cozy,1404518400
4,4,1776,B001A06VJ8,"[0, 1]",4,I did not expect this type of book to be in li...,"12 31, 2012",A3C9V987IQHOQD,Rjostler,Book,1356912000


In [ ]:
data = data[['reviewText','rating']]

In [ ]:
data.head()

,reviewText,rating
0,"Jace Rankin may be short, but he's nothing to ...",3
1,Great short read. I didn't want to put it dow...,5
2,I'll start by saying this is the first of four...,3
3,Aggie is Angela Lansbury who carries pocketboo...,3
4,I did not expect this type of book to be in li...,4


In [ ]:
data.isnull().sum()

reviewText    0
rating        0
dtype: int64

In [ ]:
data.shape

(12000, 2)

In [ ]:
data['rating'].unique()

array([3, 5, 4, 2, 1])

In [ ]:
data['rating'].value_counts()

rating
5    3000
4    3000
3    2000
2    2000
1    2000
Name: count, dtype: int64

As we see dataset not have any missing values and also it is not imbalanced

In [ ]:
## preprocessing and cleaning

In [ ]:
# positive review is 1 and negative review is 0
data['rating'] = data['rating'].apply(lambda x:0 if x<3 else 1)

In [ ]:
data.head()

,reviewText,rating
0,"Jace Rankin may be short, but he's nothing to ...",1
1,Great short read. I didn't want to put it dow...,1
2,I'll start by saying this is the first of four...,1
3,Aggie is Angela Lansbury who carries pocketboo...,1
4,I did not expect this type of book to be in li...,1


In [ ]:
data['rating'].unique()

array([1, 0])

In [ ]:
## 1. lower all the cases
data['reviewText']=data['reviewText'].str.lower()

In [ ]:
# removing special characters
data['reviewText'] = data['reviewText'].apply(lambda x: re.sub('[^a-z A-Z 0-9-]+','',x))

In [ ]:
data['reviewText']

0        jace rankin may be short but hes nothing to me...
1        great short read  i didnt want to put it down ...
2        ill start by saying this is the first of four ...
3        aggie is angela lansbury who carries pocketboo...
4        i did not expect this type of book to be in li...
                               ...                        
11995    valentine cupid is a vampire- jena and ian ano...
11996    i have read all seven books in this series apo...
11997    this book really just wasnt my cuppa  the situ...
11998    tried to use it to charge my kindle it didnt e...
11999    taking instruction is a look into the often hi...
Name: reviewText, Length: 12000, dtype: object

In [ ]:
from nltk.corpus import stopwords

In [ ]:
## remove stopwords
nltk.download('stopwords')
data['reviewText'] = data['reviewText'].apply(lambda x: " ".join([y for y in x.split() if y not in stopwords.words('english')]))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
# remove urls
data['reviewText'] = data['reviewText'].apply(lambda x: re.sub(r'(http|https|ftp|ssh)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?','',str(x)))

In [ ]:
# remove html tags
!pip install beautifulsoup4
from bs4 import BeautifulSoup
data['reviewText'] = data['reviewText'].apply(lambda x: BeautifulSoup(x,'lxml').get_text())

In [ ]:
# removing any additional spaces
data['reviewText'] = data['reviewText'].apply(lambda x: " ".join(x.split()))

In [ ]:
data.head()

,reviewText,rating
0,jace rankin may short hes nothing mess man hau...,1
1,great short read didnt want put read one sitti...,1
2,ill start saying first four books wasnt expect...,1
3,aggie angela lansbury carries pocketbooks inst...,1
4,expect type book library pleased find price right,1


In [ ]:
# apply lematizer
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()


In [ ]:
def lemmatizer_words(text):
  return " ".join([lemmatizer.lemmatize(word)for word in text.split()])

In [ ]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
data['reviewText'] = data['reviewText'].apply(lambda x: lemmatizer_words(x))

In [ ]:
data.head()

,reviewText,rating
0,jace rankin may short he nothing mess man haul...,1
1,great short read didnt want put read one sitti...,1
2,ill start saying first four book wasnt expecti...,1
3,aggie angela lansbury carry pocketbook instead...,1
4,expect type book library pleased find price right,1


In [ ]:
# train test split
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(data['reviewText'],data['rating'],test_size=0.20)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
bow = CountVectorizer()
tfidf = TfidfVectorizer()

In [ ]:
X_train_bow = bow.fit_transform(X_train).toarray()
X_test_bow = bow.transform(X_test).toarray()

In [ ]:
X_train_tfidf = tfidf.fit_transform(X_train).toarray()
X_test_tfidf = tfidf.transform(X_test).toarray()

In [ ]:
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score,classification_report

In [ ]:
nb_model_bow= GaussianNB().fit(X_train_bow,y_train)
nb_model_tfidf = GaussianNB().fit(X_train_tfidf,y_train)

In [ ]:
y_pred_bow = nb_model_bow.predict(X_test_bow)
y_pred_tfidf = nb_model_tfidf.predict(X_test_tfidf)

In [ ]:
print(accuracy_score(y_test,y_pred_bow))
print(accuracy_score(y_test,y_pred_tfidf))

0.5829166666666666
0.58375
